# This code file is to read jason data and create appropriate format

In [1]:
import json
import re
import pandas as pd
import nltk

In [2]:
def count_word_char(line):
    '''
    For simplexity we can first ignore special character
    
    1. Split line if it is not a word character
    2. Iterated each word, count if it is one word, count total characters
    '''
    
    words = [word for word in re.split(r'\W', line) if word.isalnum()] # divide string based on symbol
    
    symbols = [word.strip() for word in re.split(r'\w', line) if word not in ['', ' ']] # devide string based on word, only keep symbol not '' or ' '
    
    wc, cc = 0, 0 # counting 
    word_dict = {} # word dictionary
    
    for word in words: # For each word in the word list
        
        cc += len(word) # add current word characters length
        
        if not word.isdecimal(): # If the word is a not a pure number
            wc += 1 # counter + 1
            
            # Save word frequency
            
            if word in word_dict: 
                word_dict[word] += 1
            else:
                word_dict[word] = 1  
    print("""Symbols are: {}\nTotal words count: {}\nTotal character count: {}\n""".format(symbols, wc, cc))
    
    print("-"*31)
    print("{0:15}|{1:>15}".format("Word", "Frequency"))
    print("-"*31)
    for word, frequency in word_dict.items():
        print("{0:15}|{1:>15}".format(word, frequency))
    
    return symbols, wc, cc, word_dict

In [3]:
file_name = 'data/review_train.json'
review_train = []
with open(file_name, 'r') as f:
    try:
        while True:
            line = f.readline()
            if line:
                r = json.loads(line)
                review_train.append(r)
            else:
                break
    except:
        f.close()

In [7]:
review_example = review_train[1]
review_example
type(review_example)
review_example['text']
_, _, _, _ =count_word_char(review_example['text'])

Symbols are: ['*', '*', "'", '!', "'", ';', ',', '!', "'", '-', '.', '--', '---', '.', '(', ')', '.', '!', '...', ',', ',', '-', '.', ',', "'", "'", '.', '!', '.', '-', '.', "'", "'", '--', "'", '.', '&', '.', '?', '!', ',', "'", '.', '!']
Total words count: 282
Total character count: 1218

-------------------------------
Word           |      Frequency
-------------------------------
I              |              5
adore          |              1
Travis         |              6
at             |              1
the            |             13
Hard           |              1
Rock           |              1
s              |              5
new            |              2
Kelly          |              1
Cardenas       |              1
Salon          |              1
m              |              2
always         |              1
a              |              7
fan            |              1
of             |              4
great          |              1
blowout        |              5
and 

In [25]:
tokens = count_word_char(review_example['text'])[3]

Symbols are: ['*', '*', "'", '!', "'", ';', ',', '!', "'", '-', '.', '--', '---', '.', '(', ')', '.', '!', '...', ',', ',', '-', '.', ',', "'", "'", '.', '!', '.', '-', '.', "'", "'", '--', "'", '.', '&', '.', '?', '!', ',', "'", '.', '!']
Total words count: 282
Total character count: 1218

-------------------------------
Word           |      Frequency
-------------------------------
I              |              5
adore          |              1
Travis         |              6
at             |              1
the            |             13
Hard           |              1
Rock           |              1
s              |              5
new            |              2
Kelly          |              1
Cardenas       |              1
Salon          |              1
m              |              2
always         |              1
a              |              7
fan            |              1
of             |              4
great          |              1
blowout        |              5
and 

In [39]:
from nltk.corpus import stopwords
clean_tokens = dict()
sr = stopwords.words('english')
print(tokens)
for token in tokens:
    if not token in sr:
        clean_tokens.setdefault(token, []).append(tokens[token])

{'I': 5, 'adore': 1, 'Travis': 6, 'at': 1, 'the': 13, 'Hard': 1, 'Rock': 1, 's': 5, 'new': 2, 'Kelly': 1, 'Cardenas': 1, 'Salon': 1, 'm': 2, 'always': 1, 'a': 7, 'fan': 1, 'of': 4, 'great': 1, 'blowout': 5, 'and': 9, 'no': 1, 'stranger': 1, 'to': 5, 'chains': 1, 'that': 5, 'offer': 1, 'this': 2, 'service': 1, 'however': 1, 'has': 3, 'taken': 1, 'flawless': 1, 'whole': 1, 'level': 1, 'greets': 1, 'you': 3, 'with': 3, 'his': 3, 'perfectly': 3, 'green': 1, 'swoosh': 1, 'in': 5, 'otherwise': 1, 'styled': 1, 'black': 1, 'hair': 4, 'Vegas': 2, 'worthy': 1, 'rockstar': 1, 'outfit': 1, 'Next': 2, 'comes': 1, 'most': 1, 'relaxing': 1, 'incredible': 1, 'shampoo': 2, 'where': 2, 'get': 2, 'full': 1, 'head': 1, 'message': 1, 'could': 1, 'cure': 1, 'even': 1, 'very': 1, 'worst': 1, 'migraine': 1, 'minutes': 1, 'scented': 1, 'room': 1, 'freakishly': 1, 'strong': 1, 'fingers': 1, 'good': 1, 'way': 3, 'use': 1, 'perfect': 1, 'amount': 1, 'pressure': 1, 'That': 2, 'was': 4, 'superb': 1, 'Then': 1, 'sta

In [40]:
clean_tokens

{'I': [5],
 'adore': [1],
 'Travis': [6],
 'Hard': [1],
 'Rock': [1],
 'new': [2],
 'Kelly': [1],
 'Cardenas': [1],
 'Salon': [1],
 'always': [1],
 'fan': [1],
 'great': [1],
 'blowout': [5],
 'stranger': [1],
 'chains': [1],
 'offer': [1],
 'service': [1],
 'however': [1],
 'taken': [1],
 'flawless': [1],
 'whole': [1],
 'level': [1],
 'greets': [1],
 'perfectly': [3],
 'green': [1],
 'swoosh': [1],
 'otherwise': [1],
 'styled': [1],
 'black': [1],
 'hair': [4],
 'Vegas': [2],
 'worthy': [1],
 'rockstar': [1],
 'outfit': [1],
 'Next': [2],
 'comes': [1],
 'relaxing': [1],
 'incredible': [1],
 'shampoo': [2],
 'get': [2],
 'full': [1],
 'head': [1],
 'message': [1],
 'could': [1],
 'cure': [1],
 'even': [1],
 'worst': [1],
 'migraine': [1],
 'minutes': [1],
 'scented': [1],
 'room': [1],
 'freakishly': [1],
 'strong': [1],
 'fingers': [1],
 'good': [1],
 'way': [3],
 'use': [1],
 'perfect': [1],
 'amount': [1],
 'pressure': [1],
 'That': [2],
 'superb': [1],
 'Then': [1],
 'starts': [1

In [55]:
review_train_df = pd.DataFrame(review_train)

In [61]:
print(review_train_df.head())

   business_id                 date  stars  \
0        31292  2013-05-07 04:34:36    1.0   
1        35344  2017-01-14 21:30:33    5.0   
2       152538  2016-11-09 20:09:03    5.0   
3        71871  2018-01-09 20:56:38    5.0   
4        64913  2018-01-30 23:07:38    1.0   

                                                text  
0  Total bill for this horrible service? Over $8G...  
1  I *adore* Travis at the Hard Rock's new Kelly ...  
2  I have to say that this office really has it t...  
3  Went in for a lunch. Steak sandwich was delici...  
4  Today was my second out of three sessions I ha...  


In [ ]:
from nltk.stem.porter import *
stemmer = PorterStemmer()

 = tokenized_tweet.apply(lambda x: [stemmer.stem(i) for i in x]) 

In [9]:
freq.plot(20, cumulative=False)

<Figure size 640x480 with 1 Axes>